<a href="https://colab.research.google.com/github/Ibraheem101/Data-Science-learning/blob/main/huggingface/hf_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Behind the Scenes of the pipeline() function